# twitter python lib outdated with the new Twitter v2 API !

# https://twitter.com/argobot84

# https://github.com/euroargodev/floatoftheday

# Import

In [1]:
import os
from pathlib import Path
import tweepy
from ftplib import FTP
from shutil import move

import xarray as xr
import pandas as pd
import numpy as np
import gsw
import argopy
from argopy import DataFetcher as ArgoDataFetcher
from argopy.stores import ArgoIndex, httpstore
from argopy.related import load_dict, mapp_dict, ArgoNVSReferenceTables
# argopy.set_options(ftp='ftp://ftp.ifremer.fr/ifremer/argo')

import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt
import matplotlib.ticker as mticker
# from matplotlib import image
from PIL import Image, ImageFont, ImageDraw
import textwrap

import cmocean
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.crs as ccrs
import cartopy.feature as cfeature
land_feature = cfeature.NaturalEarthFeature(category='physical', name='land',
                                                scale='50m', facecolor=[0.4, 0.6, 0.7])
import seaborn as sns
sns.set_style("whitegrid")
SMALL_SIZE = 8
MEDIUM_SIZE = 10
BIGGER_SIZE = 12
plt.rc('font', size=MEDIUM_SIZE)  # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)  # fontsize of the axes title
plt.rc('axes', labelsize=SMALL_SIZE)  # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)  # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

import warnings; warnings.simplefilter('ignore')

/Users/gmaze/miniconda3/envs/argobot/lib/python3.8/site-packages/pyproj/__init__.py:89: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()


In [2]:
import argopy
argopy.show_versions()


SYSTEM
------
commit: None
python: 3.8.17 | packaged by conda-forge | (default, Jun 16 2023, 07:11:34) 
[Clang 14.0.6 ]
python-bits: 64
OS: Darwin
OS-release: 21.6.0
machine: x86_64
processor: i386
byteorder: little
LC_ALL: en_US.UTF-8
LANG: en_US.UTF-8
LOCALE: en_US.UTF-8
libhdf5: 1.14.2
libnetcdf: 4.9.2

INSTALLED VERSIONS: CORE
------------------------
aiohttp     : 3.8.5       
argopy      : 0.1.14      
erddapy     : 2.2.0       
fsspec      : 2023.9.1    
netCDF4     : 1.6.4       
packaging   : 23.1        
requests    : 2.31.0      
scipy       : 1.10.1      
toolz       : 0.12.0      
xarray      : 2023.1.0    

INSTALLED VERSIONS: EXT.UTIL
----------------------------
gsw         : 3.6.17      
tqdm        : 4.66.1      
zarr        : -           

INSTALLED VERSIONS: EXT.PERF
----------------------------
dask        : -           
distributed : -           
pyarrow     : -           

INSTALLED VERSIONS: EXT.PLOT
----------------------------
IPython     : 8.12.2      
carto

# Functions

In [3]:
def get_basicmetadata(wmo):
    uri = 'https://fleetmonitoring.euro-argo.eu/floats/basic/{}'.format
    data = httpstore(cache=True).open_mfjson([uri(wmo)], errors="raise", url_follow=True)
    print(data)
    meta = {'pi': '?', 'project': '?'}
    meta['pi'] = data[0]['owner']
    # if 'deployment' in data[0]:
        # if 'principalInvestigatorName' in data[0]['deployment']:
        #     meta['pi'] = data[0]['deployment']['principalInvestigatorName']
    if 'projectName' in data[0] and data[0]['projectName']:
        meta['project'] = data[0]['projectName']
    elif 'projects' in data[0]:
        meta['project'] = data[0]['projects'][0]
    return meta

def get_float_model(profiler_code):
    R8 = ArgoNVSReferenceTables().tbl(8)
    return R8[R8['altLabel']==str(profiler_code)]['prefLabel'].values[0]

def get_metadata(wmo):
    uri = 'https://fleetmonitoring.euro-argo.eu/floats/{}'.format
    data = httpstore(cache=True).open_mfjson([uri(wmo)], errors="raise", url_follow=True)
    # print(data)
    meta = {'pi': '?', 'project': '?', 'profiler_code': '?', 'profiler': '?'}

    if data[0]['owner']:
        meta['pi'] = data[0]['owner']
    elif 'deployment' in data[0]:
        if 'principalInvestigatorName' in data[0]['deployment'] and data[0]['deployment']['principalInvestigatorName']:
            meta['pi'] = data[0]['deployment']['principalInvestigatorName']

    if 'projectName' in data[0] and data[0]['projectName']:
        meta['project'] = data[0]['projectName']
    elif 'projects' in data[0]:
        meta['project'] = data[0]['projects'][0]

    for param in data[0]['configurations']['techs']:
        if param['id'] == 13:
            meta['profiler_code'] = param['value']
            try:
                meta['profiler'] = get_float_model(meta['profiler_code'])
            except:
                print(meta)
                print("%s not found in R08" % meta['profiler_code'])
                meta['profiler'] = 'Unknown'

    return meta

def download_one_random_profile_data(start, end):
    """Load a random Argo profile reported between 2 dates"""
    print("Loading profile index ...")
    # daily_df = ArgoDataFetcher(src='gdac').region([-180, 180, -90, 90, 0, 2000, start.strftime('%Y-%m-%d'), end.strftime('%Y-%m-%d')]).index
    daily_df = ArgoIndex(index_file='ar_index_this_week_prof.txt', convention='core').search_tim([-180, 180, -90, 90, DATE_start.strftime('%Y-%m-%d'), DATE_end.strftime('%Y-%m-%d')]).to_dataframe()
    
    # Gather coordinate of all profiles
    week_lon = daily_df['longitude'].values
    week_lat = daily_df['latitude'].values
    week_data = [week_lon, week_lat]

    def get_a_random_profile(this_df):
        """Randomly select one profile and download data"""
        this_df = this_df.sample(1)
        this_df['cyc'] = this_df['file'].apply(lambda x: int(x.split("/")[-1].split("_")[-1].replace(".nc","").replace("A","").replace("D","").replace("B","")))
        wmo, cyc = this_df['wmo'].values[0], this_df['cyc'].values[0]
        print("Loading profile:", wmo, cyc)

        # Load and return profile:
        try:
            ds = ArgoDataFetcher(src='gdac').profile(this_df['wmo'].values[0], this_df['cyc'].values[0]).load().data
            # raise
        except:
            ds = None
        return ds, this_df

    MaxTry, itry = 10, 0
    while itry < MaxTry:
        ds, this_df = get_a_random_profile(daily_df)
        if not ds:
            print("%i/%i: An error happened when downloading this data" % (itry+1, MaxTry))
            itry += 1
        else:
            print("%i/%i: Succeeded in downloading this data !" % (itry+1, MaxTry))            
            try:        
                ds = ds.argo.point2profile()
                break
            except:
                print("%i/%i: An error happened when processing this data" % (itry+1, MaxTry))
                itry += 1
                
    if itry == MaxTry:
        raise ValueError("All attempts (N=%i) to download data failed !" % MaxTry)

    print("Post-processing data...")
    ds = ds.isel(N_PROF=[-1])  # The last profile to date
    ds.attrs['quadrant'] = 'E' if ds['LONGITUDE'] > 0 else 'W'
    ds.attrs['hemisphere'] = 'N' if ds['LATITUDE'] > 0 else 'S'
    wmo = np.unique(ds['PLATFORM_NUMBER'])[0]
    meta = get_metadata(wmo)
    ds.attrs['float_model'] = meta['profiler']
    ds.attrs['pi_name'] = meta['pi']
    ds.attrs['project_name'] = meta['project']
    
    return week_data, this_df, ds

In [4]:
# Set of Argo logo colors:
COLORS = {'CYAN': (18/256, 235/256, 229/256), 
          'BLUE': (16/256, 137/256, 182/256), 
          'DARKBLUE': (10/256, 89/256, 162/256), 
          'YELLOW': (229/256, 174/256, 41/256),
          'DARKYELLOW': (224/256, 158/256, 37/256),
         }
argo2rgba = lambda x: tuple([int(v*255) for v  in matplotlib.colors.to_rgba(COLORS[x])])


def get_a_title(profile_df, nlines=3):
    dac = profile_df['file'].values[0].split('/')[0]
    wmo, cyc = profile_df['wmo'].values[0], profile_df['cyc'].values[0]
    lon, lat = profile_df['longitude'].values[0], profile_df['latitude'].values[0]
    when = pd.to_datetime(profile_df['date'].values[0]).strftime('%A %d of %B %Y, %H:%M:%S UTC')
    hemisphere = 'N' if lat > 0 else 'S'
    quadrant = 'E' if lon > 0 else 'W'
    where = "%0.3f%s, %0.3f%s" % (np.abs(lat), hemisphere, 
                                  np.abs(lon), quadrant,
                                 )
    if nlines == 3:        
        profile_title = "Float %s - Cycle %i\nDate: %s\nPosition: %s" % (wmo, cyc, when, where)
    elif nlines == 2:
        profile_title = "Float %s - Cycle %i\nDate: %s / Position: %s" % (wmo, cyc, when, where)
    else:
        profile_title = "Float %s - Cycle %i / Date: %s / Position: %s" % (wmo, cyc, when, where)
    return profile_title


def plot_profile(this_ds, x='TEMP', y='PRES', title='?', dpi=120):
    aspect, size = 0.6, 10
    fig, ax = plt.subplots(nrows=1, ncols=1, dpi=dpi, figsize=(aspect*size, size))
    ax.plot(this_ds[x], this_ds[y], marker='.', color=COLORS['DARKYELLOW'])
    xmin, xmax = ax.get_xlim()
    ymin, ymax = ax.get_ylim()
    ax.set_ylim([0, ymax])
    ax.invert_yaxis()
    ax.set_title(title, color=COLORS['DARKBLUE'])
    # ax.xaxis.label.set_color(COLORS['BLUE'])
    # ax.yaxis.label.set_color(COLORS['BLUE'])
    # ax.tick_params(axis='x', colors=COLORS['BLUE'])
    # ax.tick_params(axis='y', colors=COLORS['BLUE'])
    ax.grid(color=COLORS['BLUE'], linestyle='--', linewidth=.2)
    ax.tick_params(color=COLORS['BLUE'], labelcolor=COLORS['BLUE'])
    for spine in ax.spines.values():
        spine.set_edgecolor(COLORS['DARKBLUE'])
    
    ax.text(xmin+(xmax-xmin)/2, ymax, r"$%s\rightarrow$" % x, fontsize=8, color=COLORS['BLUE'], horizontalalignment='center', verticalalignment='bottom')
    ax.text(xmin+0.01*(xmax-xmin), ymax/2, r"$\leftarrow %s$" % y, fontsize=8, color=COLORS['BLUE'], rotation=90, horizontalalignment='left', verticalalignment='top')

    # print((aspect*size, size))
    # print((aspect*size*dpi, size*dpi))
    # print("Dot per inch(DPI) for the figure is: ", fig.dpi)
    # bbox = ax.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
    # print("Axis sizes are (in pixels):", bbox.width*dpi, bbox.height*dpi)
    
    
def plot_map(this_ds, this_week_data, dpi=240, title='?'):
    week_lon, week_lat = this_week_data

    aspect, size = 1, 10
    fig = plt.figure(figsize=(aspect*size, size), dpi=dpi)
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
    ax.add_feature(land_feature, color=COLORS['BLUE'], edgecolor=COLORS['CYAN'], linewidth=.1, alpha=0.3)

    plt.plot(week_lon, week_lat, '.', markersize=3, color=COLORS['YELLOW'])
    plt.plot(this_ds['LONGITUDE'], this_ds['LATITUDE'], color='r', marker='.', markersize=12, markeredgecolor='r')

    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=0.5, color=COLORS['BLUE'], alpha=0.7, linestyle=':')
    gl.xlabels_top = False
    gl.ylabels_left = False
    gl.xlocator = mticker.FixedLocator(np.arange(-180, 180 + 1, 30))
    gl.ylocator = mticker.FixedLocator(np.arange(-90, 90 + 1, 15))
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER    
    gl.xlabel_style = {'color': COLORS['BLUE'], 'fontsize': 6}
    gl.ylabel_style = {'color': COLORS['BLUE'], 'fontsize': 6}
    extent = [-179.9, 180, -80, 80]
    ax.set_extent(extent)
    ax.set_title(title, color=COLORS['DARKBLUE'])
    for spine in ax.spines.values():
        spine.set_edgecolor(COLORS['DARKBLUE'])

def add_lowerband(mfname, outfname, band_height=70, color=(255, 255, 255, 255)):
    """ Add lower band to a figure
        Parameters
        ----------
        mfname : string
            source figure file
        outfname : string
            output figure file
    """
    image = Image.open(mfname, 'r')
    image_size = image.size
    width = image_size[0]
    height = image_size[1]
    background = Image.new('RGBA', (width, height + band_height), color)
    background.paste(image, (0, 0))
    background.save(outfname)

def add_2logo(mfname, outfname, logo_height=70, font_size=12, txt_color=argo2rgba('DARKBLUE')):
    """ Add 2 logos and text to a figure
        Parameters
        ----------
        mfname : string
            source figure file
        outfname : string
            output figure file
    """

    font_path = "img/Calibri_Regular.ttf"
    lfname2 = "img/logo_argofrance.png"
    lfname1 = "img/logo_argo.png"

    mimage = Image.open(mfname)

    # Open logo images:
    limage1 = Image.open(lfname1)
    limage2 = Image.open(lfname2)

    # Resize logos to match the requested logo_height:
    aspect_ratio = limage1.size[1] / limage1.size[0]  # height/width
    simage1 = limage1.resize((int(logo_height / aspect_ratio), logo_height))

    aspect_ratio = limage2.size[1] / limage2.size[0]  # height/width
    simage2 = limage2.resize((int(logo_height / aspect_ratio), logo_height))

    # Paste logos along the lower white band of the main figure:
    box = (0, mimage.size[1] - logo_height)
    mimage.paste(simage1, box)

    box = (simage1.size[0], mimage.size[1] - logo_height)
    mimage.paste(simage2, box)

    txtA = "Argo is an international program that collects information from inside " \
           "the ocean using a fleet of robotic instruments that drift with the ocean " \
           "currents and move up and down between the surface and a mid-water level.\n" \
           "These data were collected and made freely available by the International " \
           "Argo Program and the national programs that contribute to it. " \
           "(https://argo.ucsd.edu, https://www.ocean-ops.org). The Argo Program is " \
           "part of the Global Ocean Observing System.\n" \
           "This figure was created by the Argo-France program: https://www.argo-france.fr"
    xoffset = 5 + simage1.size[0] + simage2.size[0]

    def with_ftsz(txt, ftsz=10, max_char=60, img=None):
        """Determine the pixel width of a wrapped text over max_char"""

        # Wrap text:
        txt = textwrap.fill(txt, width=max_char, break_long_words=True, replace_whitespace=True)

        # Get a font:
        this_ft = ImageFont.truetype(font=font_path, size=ftsz)  # size – The requested size, in points.

        # Print text on the image and read pixel width and height:
        draw = ImageDraw.Draw(mimage.copy())
        xy = (0,0)  # We don't care where it is in the image, we just need to draw to get the final size
        draw.text(xy, txt, font=this_ft)
        left, top, right, bottom = draw.multiline_textbbox(xy, txt, font=this_ft)
        txt_sz = (right - left, bottom - top)
        return txt, txt_sz, this_ft


    def wrap_text(caption, max_width_pixel=200, font_size=12, font_path="?", img=None):
        """Determine the maximum nb of columns for a wrap text"""
        maxc = 50
        wrapped_txt, txt_sz, ft = with_ftsz(caption, ftsz=font_size, max_char=maxc, img=img)

        too_large = txt_sz[0] > max_width_pixel
        while not too_large:
            maxc += 1
            wrapped_txt, txt_sz, ft = with_ftsz(caption, ftsz=font_size, max_char=maxc, img=img)
            too_large = txt_sz[0] > max_width_pixel
        return wrapped_txt, ft

    # Print disclaimer:
    txtA, fontA = wrap_text(txtA, font_size=font_size, 
                            max_width_pixel=mimage.size[0]-xoffset-50, font_path=font_path)
    posA = (xoffset, mimage.size[1] - logo_height - 1)
    drawA = ImageDraw.Draw(mimage)
    drawA.text(posA, txtA, txt_color, font=fontA)

    # Final save
    mimage.save(outfname) 
    

def plot_ts(this_ds, x='PSAL', y='TEMP', title='?', dpi=150):
    # with sns.axes_style("white"):
    aspect, size = 1, 10
    fig, ax = plt.subplots(nrows=1, ncols=1, dpi=dpi, figsize=(aspect*size, size))
    ax.plot(this_ds[x], this_ds[y], '.-', color=COLORS['DARKYELLOW'])

    # Add in-situ density contours
    Smin, Smax = ax.get_xlim()
    Tmin, Tmax = ax.get_ylim()
    Sdim = int(round((Smax-Smin)/0.1+1, 0))
    Tdim = int(round((Tmax-Tmin)+1, 0))
    Saxe = np.linspace(Smin, Smax, Sdim)
    Taxe = np.linspace(Tmin, Tmax, Tdim)
    Sv, Tv = np.meshgrid(Saxe, Taxe, sparse=False, indexing='ij')
    RHOi = gsw.rho(Sv, Tv, 0) - 1000
    print(np.max(RHOi), np.min(RHOi))
    # Major density contours with labels:
    cs = ax.contour(Sv, Tv, RHOi, levels=np.arange(10,40,.4), linestyle='-', linewidths=0.5, colors=[COLORS['BLUE']])
    ax.clabel(cs, fontsize=8, inline=1, fmt='%0.2f')
    # Minor density contours:
    ax.contour(Sv, Tv, RHOi, levels=np.arange(10,40,.1), linestyle='-', linewidths=0.1, colors=[COLORS['BLUE']], alpha=0.5)
    
    #
    ax.text(Smin+(Smax-Smin)/2, Tmin + 0.01*(Smax-Smin), r"$%s\rightarrow$" % x, fontsize=8, color=COLORS['BLUE'], horizontalalignment='right', verticalalignment='bottom')
    ax.text(Smin+0.005*(Smax-Smin), Tmin+(Tmax-Tmin)/2, r"$%s\rightarrow$" % y, fontsize=8, color=COLORS['BLUE'], rotation=90, horizontalalignment='left', verticalalignment='top')
    ax.grid(None)
    ax.tick_params(color=COLORS['BLUE'], labelcolor=COLORS['BLUE'])
    ax.set_title(title, color=COLORS['DARKBLUE'])
    for spine in ax.spines.values():
        spine.set_edgecolor(COLORS['DARKBLUE']) 
            

In [5]:
def tweet_this(imlist, this_ds, profile_df, caption_only=False):

    # Init the Tweeter API
    consumer_key, consumer_secret, access_token, access_token_secret = os.environ['TWITTER_SECRET'].split(':')
    
    auth = tweepy.OAuthHandler(
            consumer_key,
            consumer_secret
            )
    auth.set_access_token(
            access_token,
            access_token_secret
            )
    api = tweepy.API(auth)

    client = tweepy.Client(
        consumer_key=consumer_key, consumer_secret=consumer_secret,
        access_token=access_token, access_token_secret=access_token_secret
    )

    
    # ## Text of the tweet
    
    dac = profile_df['file'].values[0].split('/')[0]
    wmo, cyc = profile_df['wmo'].values[0], profile_df['cyc'].values[0]
    lon, lat = profile_df['longitude'].values[0], profile_df['latitude'].values[0]
    when = pd.to_datetime(profile_df['date'].values[0]).strftime('%A %d of %B %Y, %H:%M:%S UTC')
    hemisphere = 'N' if lat > 0 else 'S'
    quadrant = 'E' if lon > 0 else 'W'
    where = "%0.3f%s, %0.3f%s" % (np.abs(lat), hemisphere, 
                                  np.abs(lon), quadrant,
                                 )
        
    # when = pd.to_datetime(str(this_ds['TIME'].values[0])).strftime('%A %d of %B %Y, %H:%M:%S UTC')
    # where = "%0.3f%s, %0.3f%s" % (np.abs(this_ds['LATITUDE']), this_ds.attrs['hemisphere'], np.abs(this_ds['LONGITUDE']), this_ds.attrs['quadrant'])
    long_title = "▶ Float %s, Cycle %i 🗓%s 🗺%s 📏%i measurements 🤖%s 🔬%s 💳%s" % (
        wmo, cyc,
        when, where,
        len(this_ds['N_LEVELS']),
        "%s float" % this_ds.attrs['float_model'].split(" ")[0],
        this_ds.attrs['pi_name'].strip().title(),
        this_ds.attrs['project_name'].strip())

    # Could try to add a link to convert UTC to local, eg:
    # https://www.google.com/search?q=05%3A21%3A00+UTC+to+local
    # http://www.timebie.com/std/gmt.php?q=052100

    caption = (f'Here is the Argo profile of the last 24 hours 🎉\n'
               f'{long_title}\n'
               f'https://fleetmonitoring.euro-argo.eu/float/{wmo} '
               f'#argofloat')

    # if len(caption) > twitter.api.CHARACTER_LIMIT:
        # caption = caption[0:twitter.api.CHARACTER_LIMIT-10-1] + '\u2026'

    # print(len(caption), "/", twitter.api.CHARACTER_LIMIT, "\n", caption, "\n")

    # ## Attach figures to the tweet
    media = []
    for fname in imlist:
        media.append(api.media_upload(fname, chunked=True).media_id)

    # Post it !
    if caption_only:
        return caption
    else:
        # resp = api.PostUpdate(caption,
        #                       media=media,
        #                       latitude=float(lat),
        #                       longitude=float(lon),
        #                       display_coordinates=True)
        response = client.create_tweet(
            text=caption,
            media_ids=media

        )
        print(f"https://twitter.com/user/status/{response.data['id']}")        
        
        
        return response

# Load data

In [6]:
DATE_end = pd.to_datetime('now', utc=True)
DATE_start = DATE_end - pd.DateOffset(hours=24.0)
week_data, df, ds = download_one_random_profile_data(DATE_start, DATE_end)

Loading profile index ...
Loading profile: 2903693 147
1/10: Succeeded in downloading this data !
Post-processing data...


# Create and save figures

In [7]:
vnames = ['TEMP', 'PSAL', 'CNDC', 'DOXY', 'BBP', 'TURBIDITY', 'CHLA', 'CDOM', 'NITRATE', 'BISULFIDE', 'PH_IN_SITU_TOTAL', 'DOWNWELLING_PAR']
imlist = []
for v in vnames:
    if v in ds:
        plot_profile(ds, x=v, y='PRES', title=get_a_title(df), dpi=240)
        iname = '%s.png' % v
        plt.savefig(iname, frameon=True, bbox_inches='tight', facecolor='w', edgecolor='w')
        add_lowerband(iname, iname, band_height=100)
        add_2logo(iname, iname, logo_height=100, font_size=18)
        imlist.append(iname)

if 'TEMP' in ds and 'PSAL' in ds:
    plot_ts(ds, x='PSAL', y='TEMP', title=get_a_title(df), dpi=160)
    iname = 'TS.png'
    plt.savefig(iname, frameon=True, bbox_inches='tight', facecolor='w', edgecolor='w')
    add_lowerband(iname, iname, band_height=100)
    add_2logo(iname, iname, logo_height=100, font_size=18)
    imlist.append(iname)

plot_map(ds, week_data, dpi=120, title=get_a_title(df, nlines=2))
iname = 'MAP.png'
plt.savefig(iname, frameon=True, bbox_inches='tight', facecolor='w', edgecolor='w')
add_lowerband(iname, iname, band_height=100)
add_2logo(iname, iname, logo_height=100, font_size=12)
imlist.append(iname)
print(imlist)

27.992369993701004 21.191149620986607
['TEMP.png', 'PSAL.png', 'TS.png', 'MAP.png']


In [8]:
# ds.data_vars
df

,file,date,latitude,longitude,ocean,profiler_code,institution_code,date_update,wmo,cyc,institution,profiler
334,jma/2903693/profiles/R2903693_147.nc,2023-10-02 05:59:51,32.491,157.927,P,844,JA,2023-10-02 09:54:51,2903693,147,"JMA, Japan",Unknown


# Tweet !

In [9]:
# tweet_this(imlist, ds, df, caption_only=True)
tweet_this(imlist, ds, df)

https://twitter.com/user/status/1709188304522903960


Response(data={'edit_history_tweet_ids': ['1709188304522903960'], 'id': '1709188304522903960', 'text': 'Here is the Argo profile of the last 24 hours 🎉\n▶ Float 2903693, Cycle 147 🗓Monday 02 of October 2023, 05:59:51 UTC 🗺32.491N, 157.927E 📏86 measurements 🤖ARVOR float 🔬Jma 💳Routine-JMA\nhttps://t.co/tivbrdsQB3 #argofloat https://t.co/VQI2YDLF7Z'}, includes={}, errors=[], meta={})

***
This repository is maintained by:
<div>
<img src="https://github.com/euroargodev/euroargodev.github.io/raw/master/img/logo/logo_argofrance.png" width="200"/>
<img src="https://github.com/euroargodev/euroargodev.github.io/raw/master/img/logo/logo-lops.png" width="140"/>
</div>